# **MECS6616 Spring 2024 - Project 4**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/197115/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

**FOR PROJECT 4!!!**
- Apart from the link to your notebook, you are also required to submit the collected data file `data.pkl` and your chosen model checkpoint `dynamic.pth` to Coursework. You should put the link to your notebook in the "Comment" section of your submission.
- You can find a walkthrough of this project on Canvas under "Video Library" (03/26 Lecture), where we discuss key implementation details.

## Project Setup

In [1]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project4' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp24_project4.git

Cloning into 'mecs6616_sp24_project4'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 9.00 KiB | 1.50 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp24_project4/* /content/

In [ ]:
# Using ray for data collection is optional
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 MB 6.5 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
from collections import defaultdict
import numpy as np
class MPC:

  def __init__(self):
    # You can modify control_horizon if you want
    self.control_horizon = 10
    # Define other parameters here
    self.planning_horizon = 50


  def get_cost(self, dynamics, state, goal):
    pos_ee = dynamics.compute_fk(state)
    vel_ee = dynamics.compute_vel_ee(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel=np.linalg.norm(vel_ee)
    cost=dist+0.1*vel
    return cost,dist,vel

  def copy_action(self,dynamics,action):
    num_joints=dynamics.get_action_dim()
    action_copy=np.zeros((num_joints,1))
    for i in range(num_joints):
      action_copy[i][0]=action[i][0]
    return action_copy


  def plan_forward(self, dynamics, state, goal, action):
    states=[state]
    cost,dist,vel=self.get_cost(dynamics, state, goal)
    cost_net=dist
    for i in range(self.planning_horizon-1):
      new_state=dynamics.advance(states[-1],action)
      cost,dist,vel=self.get_cost(dynamics, states[-1], goal)
      states.append(new_state)
      cost_net+=dist
      cost_net+=0.01*vel
      #print(f'nest distance:{cost_net}')
    cost_net+=dist
    cost_net+=0.1*vel
    return cost_net


  def compute_action(self, dynamics, state, goal, action):
    # Put your code here. You must return an array of shape (num_links, 1)
    num_joints=dynamics.get_action_dim()
    #print(num_joints)
    current_action=action
    current_cost=self.plan_forward(dynamics, state, goal, current_action)
    best_action=current_action
    action_i=current_action
    best_action_i=current_action
    best_cost=self.plan_forward(dynamics, state, goal, best_action)
    best_cost_i=best_cost
    attempt=0
    while True:
      for i in range(num_joints):
        #print(f'best_cost_i:{best_cost_i}')
        action_i=self.copy_action(dynamics,best_action)
        action_i[i][0]+=(0.2)
        cost_i=self.plan_forward(dynamics, state, goal, action_i)
        #print(f'  current action_i:{action_i}, current cost_i: {cost_i}')

        if cost_i<best_cost_i:
          best_action_i=action_i
          best_cost_i=cost_i
          #print(f'best_action i updated:{best_action_i}')
      for i in range(num_joints):
        action_i=self.copy_action(dynamics,best_action)
        action_i[i][0]-=(0.2)
        cost_i=self.plan_forward(dynamics, state, goal, action_i)
        #print(f'  current action_i:{action_i}, current cost_i: {cost_i}')
        if cost_i<best_cost_i:
          best_action_i=action_i
          best_cost_i=cost_i
          #print(f'best_action i updated:{best_action_i}')
      #print(f'  final best_Action_i:{best_action_i}')
      if best_cost_i<best_cost:
        best_action=best_action_i
        best_cost=best_cost_i
        #print(f'  updated, best action: {best_action}')
      else:
        break
    # Don't forget to comment out the line below
    #raise NotImplementedError("MPC not implemented")
    #best_action = np.ones((arm.dynamics.get_action_dim(), 1))
    #print(best_action)
    return best_action

# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
action = np.zeros((arm.dynamics.get_action_dim(), 1))
# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')
    action = controller.compute_action(arm.dynamics, state, goal, action)
    arm.set_action(action)

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18
At timestep 10: Distance to goal: 3.123928189687647, Velocity of end effector: 1.4016762320849203
At timestep 20: Distance to goal: 2.5144751232810028, Velocity of end effector: 3.3348377600879275
At timestep 30: Distance to goal: 1.7533674980045943, Velocity of end effector: 4.5046200467447735
At timestep 40: Distance to goal: 1.083719739428265, Velocity of end effector: 4.501808467613845
At timestep 50: Distance to goal: 0.5895973982379236, Velocity of end effector: 3.6920387135103967
At timestep 60: Distance to goal: 0.25720863153893336, Velocity of end effector: 2.1591939983492696
At timestep 70: Distance to goal: 0.07016096809593655, Velocity of end effector: 1.0938285408792034
At timestep 80: Distance to goal: 0.11368832066579253, Velocity of end effector: 0.6308426966786052
At timestep 90: Distance to goal: 0.16072577898706855, Velocity of end effector: 0.25548445142757104
At tim

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.40824467 -0.91287255], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.69977169 -0.71436656], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.67561567 -1.57085653], Final velocity: [0.00102021]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.20550186 -1.30397198], Final velocity: [0.10349158]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.29916113 -2.36677679], Final velocity: [0.08821442]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.08941758 -1.36996846], Final velocity: [0.13498537]
score: 1.0/1.0
       
---------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step


	def forward(self, x):
		qddot = self.compute_qddot(x)
		#print(x)
		state = x[:, :2*self.num_links]
		#print(state)
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):


		# Your code goes here
		current_pos=state[:,:self.num_links]
		current_vel=state[:,self.num_links:2*self.num_links]
		next_pos=current_pos+0.01*current_vel+0.5*qddot*(0.0001)
		next_vel=current_vel+0.01*qddot
		next_state=torch.cat([next_pos,next_vel],dim=1)
		return next_state

	def compute_qddot(self, x):
		pass

class Model2Link(Model):
	def __init__(self, time_step):
		super().__init__(2, time_step)
    # Your code goes here
		self.fc1 = nn.Linear(6, 256)
		self.fc2 = nn.Linear(256, 512)
		self.fc3 = nn.Linear(512, 64)
		self.fc4 = nn.Linear(64, 2)

	def compute_qddot(self, x):
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = F.relu(self.fc3(x))
		x = self.fc4(x)
		return x

### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
import ray
# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()


def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  num_samples = 200*250
  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))
  index=0
  num_link=2
  controller=MPC()
  for i in range(200):
    arm.reset()
    initial_state = np.zeros((arm.dynamics.get_state_dim(), 1))
    initial_state[0] = -math.pi / 2.0
    arm.set_state(initial_state)
    goal = np.zeros((2,1))
    rad = np.random.uniform(low=0.05, high=1.95)
    ang = np.random.uniform(low=np.pi, high=2.0*np.pi)
    goal[0, 0] = rad*np.cos(ang)
    goal[1, 0] = rad*np.sin(ang)
    action = np.zeros((arm.dynamics.get_action_dim(), 1))
    print(f'new goal set to: [{goal[0][0]},{goal[1][0]}]')

    dt = 0.01
    time_limit = 2.5
    num_steps = round(time_limit/dt)
    action = np.zeros((arm.dynamics.get_action_dim(), 1))
    # Control loop
    for s in range(num_steps):
      if s % controller.control_horizon==0:
        state = arm.get_state()
        action = controller.compute_action(dynamics_teacher, state, goal, action)
        arm.set_action(action)
        print(f'action updated to: [{action[0][0]},{action[1][0]}]')
      X[index,:4]=arm.get_state().flatten()
      X[index,4:6]=action.flatten().reshape(1, -1)
      arm.advance()
      Y[index]=arm.get_state().flatten().reshape(1, -1)

      index+=1
      print(f'{index}th data collected')
  # ---

  return X, Y


'''
ray.shutdown()
ray.init(runtime_env={"py_modules": [robot]})
@ray.remote
def getDataSeq(goal):
  dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
  arm = Robot(dynamics_teacher)
  num_samples = 250
  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))
  index=0
  num_link=2
  controller=MPC()
  arm.reset()
  action = np.zeros((arm.dynamics.get_action_dim(), 1))
  print(f'new goal set to: [{goal[0][0]},{goal[1][0]}]')

  arm.goal = goal

  dt = 0.01
  time_limit = 2.5
  num_steps = round(time_limit/dt)
  action = np.zeros((arm.dynamics.get_action_dim(), 1))
  # Control loop
  for s in range(num_steps):
    t = time.time()
    current_state=arm.get_state()
    #print(current_state)
    arm.advance()
    new_state = arm.get_state()
    X[s][:2*num_link]=np.transpose(current_state)
    X[s][(2*num_link):]=np.transpose(action)
    Y[s][:]=np.transpose(current_state)

    print(f'{s}th in  data collected')


    if s % controller.control_horizon==0:
      state = arm.get_state()

      # Measuring distance and velocity of end effector
      pos_ee = dynamics_teacher.compute_fk(state)
      dist = np.linalg.norm(goal-pos_ee)
      vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
      #print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')
      action = controller.compute_action(arm.dynamics, state, goal, action)
      arm.set_action(action)
      print(f'action updated to: [{action[0][0]},{action[1][0]}]')
      dict_data=dict(X=X,Y=Y)
# ---

  return dict_data

def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  num_samples = 19*11*250
  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))
  index=0
  num_link=2
  controller=MPC()
  num_goals=19*11
  goals=[]
  for i in range(21):
    rad=i*0.1
    if i==0 or i==20:
      continue
    for j in range(i+1):
      if i ==0:
        ang=0
      else:
        ang=-np.pi+j*np.pi/(i)
      arm.reset()
      goal = np.zeros((2, 1))
      goal[0, 0] = rad*np.cos(ang)
      goal[1, 0] = rad*np.sin(ang)
      goals.append(goal)

  futures = [getDataSeq.remote(goal) for goal in goals]
  dict_sequence=ray.get(futures)
  index=0
  for dict_data in dict_sequence:
    print(f'load data from {index}th sample')
    X[index:250*(index+1),:]=dict_data['X']
    Y[index:250*(index+1),:]=dict_data['Y']
    index+=1


  # ---

  return X,Y
'''

'\nray.shutdown()\nray.init(runtime_env={"py_modules": [robot]})\n@ray.remote\ndef getDataSeq(goal):\n  dynamics_teacher = ArmDynamicsTeacher(\n    num_links=2,\n    link_mass=0.1,\n    link_length=1,\n    joint_viscous_friction=0.1,\n    dt=0.01)\n  arm = Robot(dynamics_teacher)\n  num_samples = 250\n  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))\n  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))\n  index=0\n  num_link=2\n  controller=MPC()\n  arm.reset()\n  action = np.zeros((arm.dynamics.get_action_dim(), 1))\n  print(f\'new goal set to: [{goal[0][0]},{goal[1][0]}]\')\n\n  arm.goal = goal\n\n  dt = 0.01\n  time_limit = 2.5\n  num_steps = round(time_limit/dt)\n  action = np.zeros((arm.dynamics.get_action_dim(), 1))\n  # Control loop\n  for s in range(num_steps):\n    t = time.time()\n    current_state=arm.get_state()\n    #print(current_state)\n    arm.advance()\n    new_state = arm.get_state()\n    X[s][:2*num_link]=np.transpose

In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(train_loader, model,optim):
  model.train()
	# ---
	# Your code goes here
  size = len(train_loader.dataset)
  num_batches = len(train_loader)
  optimizer = optim
  scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.95,patience=5)
  loss_fn = nn.MSELoss()
  train_loss=0
  for X,y in train_loader:
      #print(X)
      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss +=  loss_fn(pred, y).item()
  train_loss /= num_batches
  print(f"Train Error: \n Avg loss: {train_loss:>8f} \n")
  return train_loss

	# ---


def test(test_loader, model):
  model.eval()
	# --
	# Your code goes here
  # --
  size = len(test_loader.dataset)
  num_batches = len(test_loader)
  test_loss = 0
  loss_fn = nn.MSELoss()
  with torch.no_grad():
    for X, y in test_loader:
        pred = model(X)
        test_loss +=  loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
  return test_loss

def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  #model_path = None
  max_test_loss = 1e4
  model = Model2Link(0.01)

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  loss_min=10

  epochs = 30
  optim=torch.optim.Adam(model.parameters(),lr=0.01)
  for epoch in range(1, 1 + epochs):
    # --
    # Your code goes here
    print(f"Epoch {epoch}\n-------------------------------")
    if epoch>20:
      optim=torch.optim.SGD(model.parameters(),lr=0.01)
    train_loss=train_one_epoch(train_loader, model,optim)
    test_loss=test(test_loader, model)
    if epoch==1:
      loss_min=test_loss
    model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
    if not os.path.exists(os.path.join(model_dir, model_folder_name)):
        os.makedirs(os.path.join(model_dir, model_folder_name))
    torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
    if epoch==1:
      loss_min=test_loss
      model_path=os.path.join(model_dir, model_folder_name, 'dynamics.pth')
    if test_loss< loss_min:
      print('model updated')
      loss_min=test_loss
      model_path=os.path.join(model_dir, model_folder_name, 'dynamics.pth')
      torch.save(model.state_dict(), 'dynamics.pth')


    # --

  return model_path

In [ ]:
model_path = train_forward_model()

Epoch 1
-------------------------------
Train Error: 
 Avg loss: 0.575259 

Test Error: 
 Avg loss: 0.412684 

Epoch 2
-------------------------------
Train Error: 
 Avg loss: 0.584492 

Test Error: 
 Avg loss: 0.407309 

model updated
Epoch 3
-------------------------------
Train Error: 
 Avg loss: 0.564288 

Test Error: 
 Avg loss: 0.406456 

model updated
Epoch 4
-------------------------------
Train Error: 
 Avg loss: 0.560273 

Test Error: 
 Avg loss: 0.406575 

Epoch 5
-------------------------------
Train Error: 
 Avg loss: 0.564611 

Test Error: 
 Avg loss: 0.407128 

Epoch 6
-------------------------------
Train Error: 
 Avg loss: 0.566084 

Test Error: 
 Avg loss: 0.407053 

Epoch 7
-------------------------------
Train Error: 
 Avg loss: 0.565918 

Test Error: 
 Avg loss: 0.406956 

Epoch 8
-------------------------------


### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = Model2Link(0.01).to(device)
        self.model.load_state_dict(torch.load(model_path))
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            self.model.eval()
            x=np.r_[state,action]
            x=x.transpose()
            #x=x.tolist()
            #x=x[0]

            with torch.no_grad():
                x = torch.tensor(x,dtype=torch.float32)
                pred = self.model(x)
            # Output should be an array of shape (6,1)
            pred=pred.numpy()
            pred=np.array(pred)
            pred=pred.transpose()
            new_state = pred
            return new_state
            # ---
        else:
            return state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = model_path
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 1.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
#model_path = model_path
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Time Limit
*   MPC evaluation (Part 1): 30 minutes
*   Data Collection with MPC (Part 2): 1 hour 20 minutes
*   Training the forward dynamics model (Part 3): 40 minutes

Note: Time limits are general guidelines. While your code must run within the allotted time, you should be able to achieve full score with a much shorter runtime


Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy. Using Ray is optional